In [876]:
# Importamos librerias necesarios
import pandas as pd
import math as mt
import numpy as np
from scipy.spatial import distance

In [877]:
# Path de nuestros archivos
FILE_PATH = '/home/andy/UNSA/9 semestre/Topicos Ciencia Datos/Jupyter/Knn/'

In [878]:
# Data de ejemplo hecha en clases para comprobar procesos
movies_example = pd.read_csv(FILE_PATH + 'data.csv')
movies_example.head()

,Unnamed: 0,Blues,Broken,Deadmau,Norah,Phoenix,Slightly,Strokes,Vampire
0,angelica,3.5,2.0,NaN,4.5,5.0,1.5,2.5,2.0
1,bill,2.0,3.5,4.0,NaN,2.0,3.5,NaN,3.0
2,chan,5.0,1.0,1.0,3.0,5.0,1.0,NaN,NaN
3,dan,3.0,4.0,4.5,NaN,3.0,4.5,4.0,2.0
4,hailey,NaN,4.0,1.0,4.0,NaN,NaN,4.0,1.0


In [879]:
# Obtenemos los valores
movies_example = movies_example.values[:,1:]

In [880]:
# Cargamos las peliculas
movies = pd.read_csv(FILE_PATH + 'ml-latest-small/movies.csv',usecols=['movieId','title'])
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [881]:
# Cargamos las puntuaciones
ratings = pd.read_csv(FILE_PATH + 'ml-latest-small/ratings.csv',usecols=['userId','movieId','rating'])
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [882]:
# Creamos nuestra matriz donde relacionaremos cada usuario con la puntuacion que le dio a una pelicula
movies_users = ratings.pivot(index='userId', columns='movieId', values='rating')#.fillna(False)
print(movies_users.shape)
movies_users.head(3)

(610, 9724)


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [883]:
# Obtenemos los valores
movies_users = movies_users.values
movies_users

array([[4. , nan, 4. , ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, nan, nan]])

In [884]:
# Tiempo de ejecucion para hallar interseccion con numpy
%time
a=np.logical_not(np.isnan(movies_users[0].astype(float)))
b=np.logical_not(np.isnan(movies_users[1].astype(float)))
intersection = a*b
print(intersection)
intersection = movies_users[0][intersection]+movies_users[1][intersection]
intersection

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
[False False False ... False False False]


array([9., 9.])

In [854]:
# Tiempo de ejecucion para hallar interseccion con bucle for
%time
a = []
for i in range(movies_users.shape[1]):
    if not np.isnan(movies_users[0,i]) and not np.isnan(movies_users[1,i]):
        a.append(movies_users[0,i]+movies_users[1,i])
print(a)

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 6.91 µs
[9.0, 9.0]


In [885]:
"""
    Entrada:
        Matrix: Matriz de datos
        User_a: Primer usuario
        User_b: Segundo usuario

    Funcion: Hallamos los valores que tienen en comun un usuario a y b.

    Salida: Tupla con los datos de interseccion para cada usuario
"""
def intersection(matrix,user_a,user_b):
    # Obtenemos los valores que no son nan
    x=np.logical_not(np.isnan(matrix[user_a].astype(float)))
    y=np.logical_not(np.isnan(matrix[user_b].astype(float)))
    # Operacion and
    intersection = x * y
    # Verificamos que existe interseccion
    if len(matrix[user_a][intersection])==0:
        return None
    # Retornamos interseccion
    return (matrix[user_a][intersection],matrix[user_b][intersection])

"""
    Entrada:
        Matrix: Matriz de datos.

    Funcion: Hallamos las intersecciones de todos los usuarios.

    Salida: Una matriz con las intersecciones de todos los usuarios.
        matriz[][]=tupla(x,y)
"""
def get_intersections(matrix):
    # Numero de usuarios
    n_user = matrix.shape[0]
    intersections = np.full((n_user,n_user),None)
    # Hallamos todas las intersecciones
    for i in range(n_user):
        # Calculamos solo ma triangular superior
        for j in range(i+1,n_user):
            result = intersection(matrix,i,j)
            # Completamos la matriz para realizar querys mas facil
            intersections[i][j] = result
            intersections[j][i] = result
    return intersections

"""
    Entrada:
        intersections: Matriz de intersecciones
        function: Funcion de distancia

    Funcion: Hallamos todas las distancias de los usuarios segun la funcion ingresada.

    Salida: Matriz de tuplas, con la distancia y el id del usuario.
"""
def get_distances(intersections,function):
    # Numero de usuarios
    n_user = intersections.shape[0]
    distances = np.full((n_user,n_user),None, dtype=object)
    # Todas las distancias, solo con intersecciones
    for i in range(n_user):
        for j in range(i,n_user):
            # Usamos la funcion de distancia entregada
            if intersections[i][j]:
                result = function(intersections[i][j])
            # En caso no existe interseccion se asigna un valor infinito.
            # Esto hara que se vayan al final al ordenarlas
            else:
                result = np.inf
            distances[i][j] = (result,j+1)
            distances[j][i] = (result,i+1)
    return distances

In [871]:
"""
    Entrada:
        intersection: Tupla de matrices con intersecciones

    Funcion: Hallamos la distancia Euclidiana

    Salida: Distancia euclidiana
"""
def euclidean_distance(intersection):
    a = intersection[0]
    b = intersection[1]
    return distance.euclidean(a, b)

"""
    Entrada:
        intersection: Tupla de matrices con intersecciones

    Funcion: Hallamos la Corelacion de pearson

    Salida: Corelacion de pearson
"""
def pearson_correlation(intersection):
    a = intersection[0]
    b = intersection[1]
    n = len(a)
    return (np.sum(a*user2)-(np.sum(a)*np.sum(b)/n))/(mt.sqrt(np.sum(a**2)-np.sum(a)**2/n)*mt.sqrt(np.sum(b**2)-np.sum(b)**2/n))

"""
    Entrada:
        intersection: Tupla de matrices con intersecciones

    Funcion: Hallamos la similitud del coseno

    Salida: similitud del coseno
"""
def coseno(intersection):
    a = intersection[0]
    b = intersection[1]
    return a*b/abs(a)*abs(b)

"""
    Entrada:
        intersection: Tupla de matrices con intersecciones

    Funcion: Hallamos la distancia de manhattan

    Salida: distancia de manhattan
"""
def manhattan_distance(intersection):
    a = intersection[0]
    b = intersection[1]
    total = 0
    for x,y in zip(a,b):
        result= abs(x-y)
        total += result
    return total


In [872]:
"""
    Entrada:
        distances: Matriz de distancias ya precalculadas
        user: Id del usuario al cual hallaremos sus vecinos
        k: Cantidad de vecinos

    Funcion: Hallamos los k vecinos mas cercanos

    Salida: Array de vecinos
"""

def knn(distances,user,k):
    # Creamos una lista con las distancias
    distance = list(distances[user-1])
    # Ordenamos los valores en tiempo O(n log(n))
    distance.sort()
    # Retornamos los  k primeros
    return distance[:k]

"""
    Entrada:
        matrix: Matriz de datos con usuarios/ peliculas = calificacion
        knn: Array con los vecinos mas cercanos
        umbral: Calificacion minima

    Funcion: Hallamos las peliculas de los vecinos mas cercanos que pasen el umbral 

    Salida: Array de tuplas, con id y puntuacion
"""
def recomendation_umbral(matrix,knn,umbral):
    # Obtenemos id del usuario restamos 1 porque la matriz inicia en 0
    user = knn[1]-1
    # Hallamos las calificaciones no nulas
    not_nan=np.logical_not(np.isnan(matrix[user]))
    # Hallamos los que pasen el umbral
    pass_umbral = matrix[user] >= umbral
    # Operacion and
    result = not_nan * pass_umbral
    return np.vstack((np.where(result), matrix[user][result])).T

"""
    Entrada:
        matrix: Matriz de datos con usuarios/ peliculas = calificacion
        user: Id del usuario
        n: Cantidad minima de peliculas a mostrar

    Funcion: Hallamos las peliculas de que no vio el usuario

    Salida: Array de n id de las peliculas que no vio
"""
def show_movies_to_see(matrix,user,n):
    movies = []
    for i in range(len(matrix[user-1])):
        # Verificamos que no la haya visto
        if np.isnan(matrix[user-1][i]):
            movies.append(i)
    # Retornamos los id de las peliculas
    return np.array(movies)[:n]

"""
    Entrada:
        matrix: Matriz de datos con usuarios/ peliculas = calificacion
        user: Id del usuario
        knn: K vecinos mas cercanos
        min_users: Cantidad minima de usuarios que si vieron la pelicula

    Funcion: Hallamos las peliculas no vio el usuario y tienen una mayor cantidad de otros
    usuarios que si las vieron.
    Salida: Array de id de las peliculas recomendadas
"""
def get_recomendations(matrix,user,knn,min_users):
    # K vecimos mas cercanos
    knn = np.array(knn,dtype=int)
    # Obtenemos los id de los vecinos
    neighbors = knn[:,1]
    # Obtenemos las peliculas de los vecinos
    movie_neighbors = np.take(matrix,neighbors)
    # Obtenemos las peliculas que no vio el usuario
    not_see = show_movies_to_see(matrix,user-1,-1)
    # Recorremos las peliculas que no vio
    for i in not_see:
        count = 0
        # Comprobamos cuantos vecinos si la vieron
        for j in neighbors:
            if not np.isnan(matrix[j,i]):
                count +=1
        # Si supera el minimo, lo mostramos
        if count>=min_users:
            print(i+1,end=" ")
    print()

"""
    Entrada:
        matrix: Matriz de datos con usuarios/ peliculas = calificacion
        user: Id del usuario
        knn: K vecinos mas cercanos
        movie: Pelicula de la que predeciremos su calificacion

    Funcion: Hallamos una calificacion aproximada de la pelicula, segun el grado de influencia

    Salida: Puntuacion aproximada que daria el usuario
"""
def recomendation_influence(matrix,user,knn,movie):
    # Castemos a enteros los id de los vecinos
    knn = np.array(knn,dtype=int)
    # Obtenemos los id de los cevinos
    neighbors = knn[:,1]
    total = 0
    # Hallamos la distancia total
    for i in neighbors:
        total += distances[user,i][0]
    influence = 0
    # Calculamos el grado de influencia con la formula S(rating*distance/total)
    for i in neighbors:
        if not np.isnan(matrix[i,movie-1]):
            influence += (matrix[i,movie-1]*distances[user-1,i][0]/total)
    return influence
    

In [886]:
# Obtemos la matriz de intersecciones con la distancia euclidiana
%time
intersections=get_intersections(movies_users)
distances = get_distances(intersections,euclidean_distance)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.44 µs


In [874]:
# Sugerimos algunas peliculas basandonos en el umbral
%time
k = 10
kneighbors = knn(distances,580,k)
print("\n{} VECINOS CERCANOS\n".format(k))
print(kneighbors)
print("\nPOSIBLES RECOMENDACIONES\n")
print(recomendation_umbral(movies_users,kneighbors[0],4))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs

10 VECINOS CERCANOS

[(0.7071067811865476, 207), (1.0, 358), (1.224744871391589, 550), (1.224744871391589, 575), (1.3228756555322954, 194), (1.3228756555322954, 324), (1.3228756555322954, 505), (1.4142135623730951, 595), (1.5, 12), (1.5, 547)]

POSIBLES RECOMENDACIONES

[[1009.     4. ]
 [1034.     4. ]
 [1495.     4. ]
 [2144.     5. ]
 [2218.     4.5]
 [2252.     4. ]]


In [797]:
# Hallamos peliculas que no vio, algunas recomendaciones con mas personas que si vieron la pelicula
# Predecimos la puntuacion que le daria el usuario
%time
k = 10
n = 10
user = 0
movie = 1284
kneighbors = knn(distances,user,k)
print("\n {} VECINOS CERCANOS \n".format(k))
print(kneighbors)
print("\n {} PRIMERAS PELICULAS NO VISTAS \n".format(n))
movies_to_see = show_movies_to_see(movies_users,user,n)
print(movies_to_see)
print("\n POSIBLES PELICULAS A ANALIZAR \n")
get_recomendations(movies_users,user,kneighbors,5)
print("\n POSIBLES PUNTUACION DE LA PELICULA {}\n".format(movie))
print(recomendation_influence(movies_users,user,kneighbors,movie))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs

 10 VECINOS CERCANOS 

[(0.5, 53), (0.5, 429), (0.5, 508), (0.5, 545), (1.0, 360), (1.0, 575), (1.118033988749895, 158), (1.3228756555322954, 92), (1.5, 35), (1.5, 289)]

 10 PRIMERAS PELICULAS NO VISTAS 

[ 1  2  3  4  6  7  8  9 10 11]

 POSIBLES PELICULAS A ANALIZAR 

1284 2145 

 POSIBLES PUNTUACION DE LA PELICULA 1284

3.142824427387853


In [888]:
"""
    Esta funcion se encargara de pedir los datos al usuario y luego agregarlo a la matriz principal
"""

def insert_new_user():
    print("BIENVENIDO!")
    input_data = input("Deseas ver el listado de peliculas?(Y/N)")
    while(True):
        if input_data=='Y':
            movies.head(100)
        elif input_data=='N':
            break
        else:
            input_data = input("Deseas ver el listado de peliculas?(Y/N)")
    input_data = input("Ingresa las pelicula que ya viste: ")
    new_movies = np.array(input_data.split(),dtype=int)
    print("Ingresaste {} peliculas".format(len(new_movies)))
    input_data = input("Ingresa la puntuacion que le diste: ")
    new_ratings = np.array(input_data.split(),dtype=float)
    new_user = np.full((movies_users.shape[1]),np.nan)
    for movie,rating in zip(new_movies,new_ratings):
        new_user[movie-1] = rating
    np.append(movies_users,new_user)
"""
    Esta funcion se encargara de recomendar peliculas y predecir calificaciones
"""

def recommend_user():
    print("Te recomendamos las siguientes peliculas!")
    kneighbors = knn(distances,user,20)
    
    input_data = float(input("Define una puntuacion minima: "))
    print(recomendation_umbral(movies_users,kneighbors[0],input_data))
    print("Recomendadas por otros usuarios que vieron las mismas peliculas")
    get_recomendations(movies_users,-1,kneighbors,5)
    print("Escoge una pelicula para ver adivinar cuanto te gustara")
    while(True):
        input_data = int(input())
        print(recomendation_influence(movies_users,-1,kneighbors,input_data))

In [889]:
insert_new_user()

BIENVENIDO!
Deseas ver el listado de peliculas?(Y/N)N
Ingresa las pelicula que ya viste: 1992  999 922  1233 89 818 500 392 102 1800 123 1320 400 283 730 293 847 902 816 183
Ingresaste 20 peliculas
Ingresa la puntuacion que le diste: 4 5 4 3 5 4 2 3 5 3.5 4 3.5 4.0 1 5 1 4 2 2 1


In [ ]:
4 5 4 3 5 4 2 3 5 3.5 4 3.5 4.0 1 5 1 4 2 2 1

In [ ]:
1992  999 922  1233 89 818 500 392 102 1800 123 1320 400 283 730 293 847 902 816 183

In [ ]:
recommend_user()

Te recomendamos las siguientes peliculas!
Define una puntuacion minima: 4
[[ 171.    5.]
 [ 213.    5.]
 [ 338.    5.]
 [ 357.    5.]
 [ 419.    5.]
 [ 601.    5.]
 [ 668.    5.]
 [ 698.    5.]
 [ 704.    5.]
 [ 801.    5.]
 [ 837.    5.]
 [ 853.    5.]
 [ 954.    5.]
 [ 995.    5.]
 [1107.    5.]
 [1458.    5.]
 [1970.    5.]
 [2014.    5.]
 [2340.    5.]
 [2999.    5.]]
Recomendadas por otros usuarios que vieron las mismas peliculas
899 900 1044 1055 1057 
Escoge una pelicula para ver adivinar cuanto te gustara
1057
0.2932803102971471
1055
0.4004474919269141
1044
0.5854418754866733


In [770]:
movies = pd.read_csv(FILE_PATH + 'ml-latest/movies.csv',usecols=['movieId','title'])
ratings = pd.read_csv(FILE_PATH + 'ml-latest/ratings.csv',usecols=['userId','movieId','rating'])
#movies_users = ratings.pivot(index='userId', columns='movieId', values='rating').unstack()
#movies_users = ratings.groupby(['userId','movieId'])['rating'].max().unstack()
n = int(ratings.shape[0]/2)
subset1 = ratings[:n] 
subset2 = ratings[n:]
matrix1 = subset1.pivot_table(index='userId', columns='movieId', values='rating')
matrix2 = subset2.pivot_table(index='userId', columns='movieId', values='rating')
complete_matrix = matrix1.join(matrix2)
movies_users = movies_users.values
print(movies_users.shape)

ValueError: Unstacked DataFrame is too big, causing int32 overflow